<a href="https://colab.research.google.com/github/carlibeisel/CDL_analyze_croptype_by_pixel/blob/main/BPBC_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Climate + LULCC for Boise Project Board Control locations

##LULCC##

In [1]:
## IMPORT PACKAGES ##
import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats==2.1.3 #very important
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os
import matplotlib.lines as lines
import matplotlib.patches as patch

In [2]:
#Navigate to your directory

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
## ------------------------------ ##
## SUBSET GEOSPATIAL DATA TO BPBC ##
## ------------------------------ ##

import rasterio
with rasterio.Env(GTIFF_SRS_SOURCE='EPSG'): #added to project all in same EPSG coordinate system
    shp_file = gpd.read_file('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/shapefile_bpbc/BPBC_Divisions-IDTM.shp') #open shapefile
    names = shp_file['DivisionNa']
    files = glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/*.tiff') #get all the years of cdl imagery
    data =[]
    for i in range(len(files)):
        data.append(rasterio.open(files[i])) #open cdl image and append to a list
        print(rasterio.open(files[i])) #added to check
    shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
    years = np.arange(1987, 2021) #years of LCMAP data
    collection = []
    for i in range(len(shp)):
        for n in range(len(years)):
            dataset = data[n]
            year_out = dataset.name[82:86] #change these numbers if printing name wrong due to projection error
            extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
            coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
            out_img, out_transform = mask(dataset=dataset, shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
            out_meta = dataset.meta.copy()
            out_meta.update({"driver": "GTiff",
                             "height": out_img.shape[1],
                             "width": out_img.shape[2],
                             "transform": out_transform})
            # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
            in_file = files[n]
            out_filename = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/lcmap_masked_bpbc/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
            out_tiff = rasterio.open(out_filename, 'w', **out_meta)
            out_tiff.write(np.squeeze(out_img, 0), 1)
            out_tiff.close()
            collection.append(out_img)

<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1987_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1986_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1989_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1988_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1990_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1991_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1992_V12_LCPRI.tiff' mode='r'>
<open DatasetReader name='/

In [18]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/lcmap_masked_bpbc/'+names[i]+'_*.tif')) #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years) #import all CDL rasters and mask
  temporal_group.append(sta)

In [19]:
# ------------------------------------ #
# CALCULATE THE CLASS PROPORTIONS BPBC #
# ------------------------------------ #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/proportion/'+names[i]+'_prop.csv')
  proportions.append(df)

In [20]:
# ------------------------------- #
#  CALCULATE LARGEST PATCH METRICS
# ------------------------------- #
patch = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['largest_patch_index'])
  df.to_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/largest_patch_index/'+names[i]+'_patch.csv')
  patch.append(df)

In [21]:
# ------------------------------- #
#     CALCULATE CONTAGION
# ------------------------------- #
# Original code computed contagion + largest_patch_index in the same step. BUt you
# cannot compute contagion with landscape_metrics.
# Make sure you have the correct version of PyLandStats (2.1.3) or it won't run,
# the error code won't look like its a package version issue.
for i in range(len(names)):
    contag = []
    files= sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/lcmap_masked_bpbc/'+names[i]+'_*.tif')) #name for all the csv files
    k = 0
    for file in files:
        sta = pylandstats.Landscape(file, nodata=0)
        DD = sta.contagion()
        output_file = '/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/contagion/' + names[i] + '_contagion.csv'
        contag.append({'dates': years[k], 'contagion': DD})
        k = k+1
        contag_df = pd.DataFrame(contag)
        contag_df.to_csv(output_file, index=False)

In [22]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes
files_proportions = sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/proportion/*_prop.csv'))
files_contagion = sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/contagion/*_contagion.csv'))
files_patch = sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/largest_patch_index/*_patch.csv'))
names = list(sorted(shp_file['DivisionNa']))

proportions = []
for i in files_proportions:
  data = pd.read_csv(i)
  proportions.append(data)
contagion = []
for i in files_contagion:
  data = pd.read_csv(i)
  contagion.append(data)
patch = []
for i in files_patch:
  data = pd.read_csv(i)
  patch.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)

In [23]:
## ----------------------------------------------- ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING BPBC ##
## ----------------------------------------------- ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/proportion/bpbc_change.csv')


In [24]:
## -------------------- ##
## MERGE DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  contag = contagion[i]
  pat = patch[i]
  df_merge = df.merge(contag, on='dates', how='left').merge(pat, on='dates', how='left')
  display (df_merge)
  df_merge.to_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_lulcc_out/final/'+ names[i] +'.csv')
  merged.append(df_merge)

,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,DIVISION 2,37.686249,61.116930,64.298330,51.727201
1,1988,DIVISION 2,38.622758,60.086976,67.387426,51.239436
2,1989,DIVISION 2,39.376483,59.368165,67.168970,50.912377
3,1990,DIVISION 2,40.438270,58.302784,66.947749,50.317304
4,1991,DIVISION 2,41.641765,57.177844,69.562319,48.110553
5,1992,DIVISION 2,42.716388,56.153538,66.805937,47.456948
6,1993,DIVISION 2,44.709549,54.256903,66.581798,45.094626
7,1994,DIVISION 2,47.206391,51.789840,66.633710,42.538739
8,1995,DIVISION 2,48.564431,50.437448,66.815400,41.583233
9,1996,DIVISION 2,49.797193,49.249869,67.106616,40.591274


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,DIVISION 3,8.476869,84.341901,73.091890,82.835707
1,1988,DIVISION 3,8.776901,83.865756,72.470351,82.419213
2,1989,DIVISION 3,9.002723,83.613304,72.141162,82.194101
3,1990,DIVISION 3,9.308791,83.283446,71.793535,81.861048
4,1991,DIVISION 3,9.656402,82.987321,71.524402,78.771255
5,1992,DIVISION 3,10.027092,82.618406,71.061410,78.428971
6,1993,DIVISION 3,10.761725,81.941293,70.266907,77.825357
7,1994,DIVISION 3,11.476830,81.216602,69.602392,79.857760
8,1995,DIVISION 3,11.889063,80.737616,69.150254,74.778527
9,1996,DIVISION 3,12.186609,80.353789,68.801976,74.348186


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,DIVISION 4,4.775485,82.453885,78.321802,81.599207
1,1988,DIVISION 4,4.741566,82.417288,76.445994,81.558147
2,1989,DIVISION 4,4.791552,82.386047,76.370959,81.495664
3,1990,DIVISION 4,4.822347,82.322671,76.310507,81.447463
4,1991,DIVISION 4,4.875458,82.284289,78.155187,81.435859
5,1992,DIVISION 4,4.964273,82.218235,77.967696,81.345705
6,1993,DIVISION 4,5.097273,82.155306,75.816605,81.214491
7,1994,DIVISION 4,5.149491,81.925458,75.852493,81.087293
8,1995,DIVISION 4,5.220007,81.782640,75.760457,80.974378
9,1996,DIVISION 4,5.189658,81.463084,75.484706,80.681600


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,DIVISION 5,4.017666,94.742081,89.061318,94.633387
1,1988,DIVISION 5,4.134940,94.580185,88.835271,94.451468
2,1989,DIVISION 5,4.199585,94.495518,89.590118,94.361081
3,1990,DIVISION 5,4.215603,94.451468,89.528948,94.320464
4,1991,DIVISION 5,4.229905,94.438883,89.583470,94.306734
5,1992,DIVISION 5,4.289972,94.354788,88.552573,94.232364
6,1993,DIVISION 5,4.382648,94.244950,88.382878,94.121382
7,1994,DIVISION 5,4.448436,94.202617,88.365729,94.065891
8,1995,DIVISION 5,4.515369,94.127675,87.012558,93.990378
9,1996,DIVISION 5,4.551981,94.033283,88.256026,93.889121


,dates,DivName,class1_urban,class2_crops,contagion,largest_patch_index
0,1987,NAMPA & MERIDIAN,56.263535,43.288654,63.804020,52.411744
1,1988,NAMPA & MERIDIAN,57.527348,41.922396,67.391572,53.705986
2,1989,NAMPA & MERIDIAN,58.361100,41.058723,63.742036,54.687318
3,1990,NAMPA & MERIDIAN,59.358153,40.058119,63.935710,55.815723
4,1991,NAMPA & MERIDIAN,60.316156,39.160467,64.334296,56.845234
5,1992,NAMPA & MERIDIAN,61.484626,38.013297,68.308896,58.873319
6,1993,NAMPA & MERIDIAN,63.238344,36.299136,68.969262,60.957699
7,1994,NAMPA & MERIDIAN,64.906862,34.641776,69.672798,62.793575
8,1995,NAMPA & MERIDIAN,66.760996,32.784092,70.499915,64.782611
9,1996,NAMPA & MERIDIAN,67.814343,31.738859,71.138167,65.859794


## CLIMATE ##

In [25]:
# Installs geemap package
import subprocess
!pip install geemap #added

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet') #input and create project on google earth engiine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00


In [26]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
## ------------------------------------------------------- ##
##   Import shapefile and start/end dates to clip dataset  ##
## ------------------------------------------------------- ##
!pip install pycrs

start_end = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_output/diversion_timeseries_out/se_dates.csv')
#shp_file = '/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/POUs/POUs_EDIT_060622_Merge.shp'
#subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE
bpbc = emap.shp_to_ee('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/shapefile_bpbc/BPBC_Divisions-IDTM.shp')

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(bpbc, 0, 2), {}, 'POU')
map.addLayerControl()
map

Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [33]:
## ------------------------------------------------- ##
##     IMPORT TEMP and ET FOR IRRIGATION SEASON      ##
## ------------------------------------------------- ##

## BPBC ##

# hashtagged out irrigation season precip metric

years = np.arange(1987,2021)

# Empty lists to store images
mean_max = []
et_irrig = []
mar_tmp = []
mar_prcp = []
mar_et = []

for i in range(len(years)):
  mar = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-03-01'), (str(years[i])+'-03-31'))
  m_tmp  = mar.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index': (str(years[i])+'-03')})
  m_prcp  = mar.select('prcp').map(lambda image: image.clip(subset)).mean().set({'system:index': (str(years[i])+'-03')})
  et_mar = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-03-01'), (str(years[i])+'-3-31'))
  et_m = et_mar.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])})
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  ir_tmp.append(tmp)
  mean_max.append(mxtmp)
  mar_tmp.append(m_tmp)
  mar_prcp.append(m_prcp)
  mar_et.append(et_m)
  start = start_end[start_end['Year'] == years[i]]
  print(int(start['StartDayofYear'].head(1)))
  if int(start['StartDayofYear'].head(1)) < 91:
    et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-03-01'), str(years[i])+'-10-31')
    et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters
  else:
    et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
    et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters
  et_irrig.append(et)


# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
means_max_temp = ee.ImageCollection(mean_max)

91
64
91
78
91
92
91
91
91
92
62
82
91
73
91
91
91
92
91
88
91
92
64
81
91
73
80
83
61
81
80
77
63
70


In [38]:
## ------------------------------------------------- ##
##          ANNUAL PRECIPITATION METRIC              ##
## ------------------------------------------------- ##

years = np.arange(1987, 2021)
annual_pr = []

for i in range(len(years)):
    daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-01-01'), (str(years[i])+'-12-31')) #get image collection for entire year
    pr = daymet.select('prcp').map(lambda image: image.clip(bpbc)).sum().set({'system:index': str(start_end['Year'][i])}) #select precip to analyze and sum
    annual_pr.append(pr)

annual_prcp = ee.ImageCollection.fromImages(annual_pr)

In [40]:
## ---------------------------------------------------------------------- ##
##    IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

## BPBC ##
# This will provide insight into antecedent moisture conditions for a POU.
#changed code to get full year antecedent precip - not just irigation season
years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(str(years[i] - 1) + '-01-01', str(years[i] - 1) + '-12-31') #trying to get full year
  prcp = daymet.select('prcp').map(lambda image: image.clip(bpbc)).sum().set({'system:index':str(i+1)}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map


Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [45]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# BPBC #

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/JAtemp_stats_bpbc.csv')
emap.zonal_statistics(means_max_temp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/ir_tmp_stats_bpbc.csv')
emap.zonal_statistics(ir_tmp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/annual_precip_stats_bpbc.csv')
emap.zonal_statistics(annual_prcp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/ant_precip_stats_bpbc.csv')
emap.zonal_statistics(ant_precip, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/et_bpbc.csv')
emap.zonal_statistics(et_irrig, bpbc, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/JAtemp_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/ir_tmp_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/annual_precip_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/ant_precip_stats_bpbc.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/et_bpbc.csv


In [48]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

## BPBC ##

years = np.arange(1987,2021)
annual_precip = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/annual_precip_stats_bpbc.csv')
ant_precip = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/ant_precip_stats_bpbc.csv')
JA_temp = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/JAtemp_stats_bpbc.csv')
irrig_temp = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/ir_tmp_stats_bpbc.csv')
et_irrig = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/et_bpbc.csv')

names = et_irrig['DivisionNa']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['annual_prcp'] = annual_precip.iloc[i,0:34].values
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/bpbc_climate_out/final_bpbc/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


In [64]:
shapefile_path = '/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/shapefile_bpbc/BPBC_Divisions-IDTM.shp'
gdf = gpd.read_file(shapefile_path)

# Print the names from the "names" column
for name in gdf['DivisionNa']:
    print(name)

NAMPA & MERIDIAN
DIVISION 2
DIVISION 3
DIVISION 4
DIVISION 5


In [71]:
## BPBC Merging ##
relates = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/bpbc_relate.csv')
bpbc = gpd.read_file('/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_input/shapefile_bpbc/BPBC_Divisions-IDTM.shp') #open shapefile
names = bpbc['DivisionNa']

# Dicharge data dict
key_list = list(bpbc['DivisionNa'])
dict_lookup = dict(zip(relates['Discharge'], relates['NewName']))
#bpbc['DivisionNa'] = [dict_lookup[item] for item in key_list]

# Land use data
land_bpbc = land_bpbc.drop(['Unnamed: 0', 'DivName'], axis=1)
all_land = pd.concat([land_bpbc,land])

# Land use change dict
key_list2 = list(land_bpbc['DivName'])
dict_lookup2 = dict(zip(relates['Shape'], relates['NewName']))
land_bpbc['Name'] = [dict_lookup2[item] for item in key_list2]

key_list3 = list(climate_bpbc['DIV_NAME'])
dict_lookup3 = dict(zip(relates['Shape'], relates['NewName']))
climate_bpbc['Name'] = [dict_lookup2[item] for item in key_list2]

## Flow data
bpbc = bpbc.drop('DiversionNa', axis=1)
div_bpbc = pd.concat([div, bpbc])
all_div = pd.DataFrame(div_bpbc[['Year', 'Name', 'Acre_feet']])
all_div = all_div.sort_values(by=['Name', 'Year'])

## Climate data
climate_bpbc = climate_bpbc.drop(['Unnamed: 0','DIV_NAME'], axis=1)
all_clim = pd.concat([climate_bpbc, clim])

NameError: name 'land_bpbc' is not defined

In [ ]:
## ------------------------------------ ##
## MERGE THREE FILES INTO ONE FILE BPBC ##
## ------------------------------------ ##

land_div = all_div.merge(all_land, left_on=['Year', 'Name'], right_on=['dates','Name'], how='left')
full_df = land_div.merge(all_clim, left_on=['Year','Name'], right_on=['Year', 'Name'], how='left').sort_values(by=['Name', 'Year'])
full_df = full_df.merge(hydromet, left_on='Year', right_on='Year', how='left').drop(['Unnamed: 0', 'dates'], axis=1)

# Get rid of New York data because using BPBC data
full_df = full_df[full_df['Name'] != 'New York Canal']
print(full_df['Name'].unique())
display(full_df)
## --------------------------------------- ##
## Export the full csv file for model in R ##
## --------------------------------------- ##

# Full dataframe export
out_path = '/content/drive/MyDrive/Data/pod_pou_lulcc/BPBC_Divisions/data_output/merged/bpbc_model_input.csv'
full_df.to_csv(out_path)
